### An example implementation of the CAREforMe pipeline using the WESAD dataset

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join("../care_for_me"))
sys.path.insert(0, module_path)
module_path = os.path.abspath(os.path.join(".."))
sys.path.insert(0, module_path)

import numpy as np
import pandas as pd
import wesad

from care_for_me import signals


ROOT_DIR = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforMe"
DATA_DIR = os.path.join(ROOT_DIR, "data")
WESAD_PATH = os.path.join(DATA_DIR, "WESAD")
METRICS = os.path.join(DATA_DIR, "metrics", "WESAD")

subject_indices = list(range(2, 12)) + list(range(13, 18))
SUBJECTS = [str(i) for i in subject_indices]

In [ ]:
# Format data to be compatible with CAREforMe pipeline
# Only needs to be run once 
# Status: COMPLETE, do not re-run
wesad.reformat_and_save_data()

In [83]:
print(wesad.generate_labels(threshold="dynamic"))

10.2
12.2
12.4
11.2
9.8
10.6
14.0
9.4
12.2
10.0
13.4
13.8
11.8
12.4
13.2
    subject  Base_STAI  TSST_STAI  Medi_1_STAI  Fun_STAI  Medi_2_STAI
0         2          0          1            0         0            0
1         3          0          1            0         0            0
2         4          0          1            1         0            1
3         5          0          1            0         1            0
4         6          0          1            0         1            0
5         7          0          1            0         0            1
6         8          0          1            1         0            0
7         9          0          1            0         1            0
8        10          0          1            0         0            0
9        11          0          1            0         0            0
10       13          1          1            0         0            1
11       14          0          1            0         0            0
12       15      

In [85]:
# Build pipeline with default preprocessing and feature extraction methods
# Asynchronous directed acyclic graph

# from care_for_me import feature_extractor
# from care_for_me import signal_acquisition
# from care_for_me import signal_preprocessor
# from care_for_me import pipeline
import biosppy as bp
import random

from care_for_me.feature_extractor.feature_extractor import FeatureExtractor
from care_for_me.signal_acquisition.signal_acquisition import SignalAcquisition
from care_for_me.signal_preprocessor.signal_preprocessor import SignalPreprocessor
from care_for_me.feature_selector.feature_selector import FeatureSelector
from care_for_me.pipeline.pipeline import Node, Pipeline

from sklearn.svm import SVC


# Change wesad_path according to your local file structure
wesad_path = "C:\\Users\\zhoux\\Desktop\\Projects\\CAREforMe\\data\\WESAD"
source_folder = os.path.join(wesad_path, "formatted")
signal_types = [
    signals.Signals.ECG,
    # signals.Signals.EDA
]
features = [
    signals.Features.HR, signals.Features.RMSSD, signals.Features.SDNN,
    # signals.Features.HR,
    signals.Features.MEAN_SCL, signals.Features.SCR_RATE
]

signal_acq = SignalAcquisition(source_folder=source_folder, signal_types=signal_types)
signal_preprocessor = SignalPreprocessor(skip=True, resample_rate=100)
feature_extractor = FeatureExtractor()

labels = [random.choice([0, 1]) for _ in range(len(SUBJECTS))]
model = SVC()
feature_selector = FeatureSelector(model, features, labels, num_features=3)

pipeline = Pipeline()

pipeline.generate_nodes_from_layers(
    [signal_acq, signal_preprocessor, feature_extractor, feature_selector]
)

pipeline.run()

Running node Signal Acquisition...
Elapsed time for Signal Acquisition: 0.0
Running node Signal Preprocessor...
Elapsed time for Signal Preprocessor: 0.39056849479675293
Running node Feature Extractor...
Elapsed time for Feature Extractor: 0.3585641384124756
Running node Feature Selector...
['subject' 'HR' 'RMSSD']
Elapsed time for Feature Selector: 0.06751298904418945
